### 브랜드 전체 키워드 리스트

In [1]:
brandkeyword = {
                   '남자 아르기닌':['울트라아르기닌','남성효모']}




In [2]:
for br in brandkeyword.keys():
    print()
    print('브랜드:  ', br)
    for keyword in brandkeyword[br]:
        print('키워드:  ', keyword)


브랜드:   남자 아르기닌
키워드:   울트라아르기닌
키워드:   남성효모


### 네이버 트래킹 함수

In [3]:
import json
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import openpyxl
import sys
import time

def crawl(keyword):
    header = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'cookie': 'nx_ssl=2; NNB=JCBOWSM5V3GGE; autocomplete=use; AD_SHP_BID=9; _ga=GA1.2.1164695733.1657581707; ncpa=95694|l5he3m5s|7918e86803866ebcebed07f9bd04a8f0e2aca26d|95694|684a7e7d3224595049682f2307a9f22892167a99:628226|l5he7pjc|8fedfc6ac1b0c11c21fdef3ef76f8590e761a269|s_2c8b6ed8b5818|41ebe434fb086f6d675ef212c1060077f45e48cf:692176|l5hf3360|32152b4df28dee72ac67aa59f9f1a81394112511|s_3384d4e2f4bc9|ad9b61abc92ca620867ffe2c6c38427095bc8d2c; NaverSuggestUse=use%26unuse; nid_inf=1733643794; NID_AUT=JLVAAKb4ILh68mRXBXne7DHBpBoC0Ity1GENP5nd7nU1dt+j7FRZbIFGScB7aNGZ; NID_JKL=zTfjaTc1rrIQGVGMNt+mHdJokT1ixuQnbq9LBxkzEzU=; spage_uid=; sus_val=XXHHJzm5614a3gxX9E8bUrbI; NID_SES=AAABo+5PzlcBGyZepMwt9TXG5g8t4O4Z5dJ19iAMO8YZej3kc80LyFQLIKGXCKzfMm6SbibKHs0/KledHneQvFogyKXUHBSWXygygeLUCXj2yNpRs3LbQTjAIPmSy6dmPn3FbqVrdsjFYf9/9zWOkLF0CGRz36HGyBVMyOmT8fnYjik0qyEaP7AEQl9SsCZbLbs7A4RmWQaSuWR6TKkhOYBmLA3l+DdkLoOrA3GjiFFUN5H17RbSTpzaXLRXTj8muFuBWFtltWXnsdTX9WsBQhM2yo8j4oybhA274H14uaLAUU0Dy63TL+2gNCEocVqGKjM/yRIfDaTZTLxplfjlBDVBBqG4IQUpMaN6Hzqgd0loVnO/tJF7mASOATTkWxJUqtDu3vVP0s0lUKVNR52b2F08jd8wrYaHgjbtv06MJyVAGnatb6/nECcIjd64tncODJVyM0RDocEGTmJQDEIllaDIPtRcmgpcFVST/crv5CckRuTFG3gbrA5MsUKro2oLJyEGRYI00hLy8uidQZL92ya5lcWWGFefhyd2tTLuin8/5sq439p8+AW2S8bjxUozcK8naA==',
    'referer': 'https://shopping.naver.com/home/p/index.naver',
    'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
    }    
    if 'mobile' in kind and 'powerlink' not in kind:
        url = "https://msearch.shopping.naver.com/search/all?sort=rel&pagingIndex={}&pagingSize=40&viewType=lst&productSet=total&query={}&origQuery={}&frm=NVSHPAG".format(
            pageNum, keyword, keyword)
    elif 'pc' in kind and 'powerlink' not in kind:
        url = "https://search.shopping.naver.com/search/all.nhn?origQuery={}&pagingIndex={}&pagingSize=40&viewType=list&sort=rel&frm=NVSHPAG&query={}".format(
            keyword, pageNum, keyword)
    elif 'mobile' in kind and 'powerlink' in kind:
        url = "https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query={}".format(keyword)
    elif 'pc' in kind and 'powerlink' in kind:
        url = "https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query={}".format(keyword)
    data = requests.get(url, headers=header)
    time.sleep(2)
    return data.content

def powergetProductInfo(ads):
    if 'pc' in kind:
        title = ads.find('div', {'class': 'lnk_tit'}).text
        url = ads.find('div', {'class': 'url_area'}).find('a', {'class':'lnk_url'}).text
        return {"keyword": keyword, "url": url, "title": title}
    elif 'mobile' in kind:
        title = ads.find('span', {'class':'tit'}).text
        url = ads.find('span', {'class':'url'}).text
        return {"keyword": keyword, "url": url, "title": title}

def getProductInfo(li):
    img = li['item']['productName']
    priceReload = li['item']['price']
    ranks = li['item']['rank']
    try:
        adsNum = li['item']['adId']
    except KeyError:
        adsNum = 'No-Ads'
    try:
        if type(li['item']['lowMallList']) == list:
            collection= li['item']['lowMallList']
            for i in range(1):
                aTit = '(가격비교묶기) ' + collection[i]['name']
                adsNum = 'No-Ads'
                id = collection[i]['mallPid']
        else:
            aTit = li['item']['mallName']
            id = li['item']['mallProductId']
    except KeyError:
        id = 'None'
    return {"ads": adsNum, "keyword": keyword, "name": img, "price": priceReload, "shopper": aTit, "productID": id, "rank": ranks}


def parse(pageString):
    if 'mobile' in kind and 'powerlink' not in kind:
        products = []
        bsObj = BeautifulSoup(pageString, 'html.parser')
        data = bsObj.findAll('script', {'type': 'application/json'})
        for datas in data:
            ojson = json.loads(datas.text)['props']['pageProps']['initialState']
            ojson = json.loads(ojson)['compositeProducts']['list']
            for li in ojson:
                product = getProductInfo(li)
                products.append(product)
            return products
    elif 'pc' in kind and 'powerlink' not in kind:
        products = []
        bsObj = BeautifulSoup(pageString, 'html.parser')
        data = bsObj.findAll('script', {'type': 'application/json'})
        for datas in data:
            ojson = json.loads(datas.string)['props']['pageProps']['initialState']['products']['list']
            for li in ojson:
                product = getProductInfo(li)
                products.append(product)
        return products
    elif 'mobile' in kind and 'powerlink' in kind:
        products = []
        bsObj = BeautifulSoup(pageString, 'html.parser')
        lis = bsObj.findAll('ul', {"id": re.compile("power_link_body")})
        for li in lis:
            ads_list = li.findAll('div', {'class':'total_wrap'})
        for ads in ads_list:
            product = powergetProductInfo(ads)
            products.append(product)
        return products
    elif 'pc' in kind and 'powerlink' in kind:
        products = []
        bsObj = BeautifulSoup(pageString, 'html.parser')
        lis = bsObj.findAll('div', {'id': 'power_link_body'})
        for li in lis:
            ads_list = li.findAll('div', {'class': 'inner'})
        for ads in ads_list:
            product = powergetProductInfo(ads)
            products.append(product)
        return products


### 쿠팡, 옥션, 11번가, 지마켓 데이터 가져오기

In [5]:
import json
import requests 
import urllib.parse
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import time
errorlst = []
for br in brandkeyword.keys():
    print()
    print('브랜드:  ', br)
    
    with pd.ExcelWriter('{} 노출 트래킹 결과_{}.xlsx'.format(br, datetime.now().strftime('%y%m%d'))) as writer:
        for jojo in range(len(brandkeyword[br])):
            keyword = brandkeyword[br][jojo]
            keyworddf = pd.DataFrame()
            print('키워드:  ', keyword)

        ######
        # 네이버 시작
        ######
            keyword = brandkeyword[br][jojo]
            naverqq = pd.DataFrame()
            result = pd.DataFrame()
            for kind in ['pc','mobile']:
                brand = br
                print(kind, keyword, " 서치중")
                pageNum = 1
                ad_rank = 1
                rank = 1
                pc_product = pd.DataFrame()
                mobile_product = pd.DataFrame()
                for loop in range(1):
                    pageString = crawl(keyword) # 키워드를 크롤링
                    products = parse(pageString)
                    for product in products:
                        if 'pc' in kind: #pc 오가닉
                            if product['ads'] != 'No-Ads':
                                pc_product = pc_product.append(pd.DataFrame([['광고', brand, keyword, pageNum, rank, product['name'], product['productID'], product['price'], product['shopper'], product['ads'], ad_rank]], columns=['구분', '브랜드', '키워드', '페이지수', '위치', '상품명', '상품ID', '가격', '셀러명', '광고아이디', '광고/오가닉순위']))
                                ad_rank += 1
                                rank += 1
                            else:
                                pc_product = pc_product.append(pd.DataFrame([['', brand, keyword, pageNum, rank, product['name'], product['productID'], product['price'], product['shopper'], product['ads'], product['rank']]], columns=['구분', '브랜드', '키워드', '페이지수', '위치', '상품명', '상품ID', '가격', '셀러명', '광고아이디', '광고/오가닉순위']))
                                rank += 1
                            if len(pc_product) > 0:
                                pc_product['분류'] = '네이버_PC'
                                result = pc_product

                        else: #모바일 오가닉
                            if product['ads'] != 'No-Ads':
                                mobile_product = mobile_product.append(pd.DataFrame([['광고', brand, keyword, pageNum, rank, product['name'], product['productID'], product['price'], product['shopper'], product['ads'], ad_rank]], columns=['구분', '브랜드', '키워드', '페이지수', '위치', '상품명', '상품ID', '가격', '셀러명', '광고아이디', '광고/오가닉순위']))
                                ad_rank += 1
                                rank += 1
                            else:
                                mobile_product = mobile_product.append(pd.DataFrame([['', brand, keyword, pageNum, rank, product['name'], product['productID'], product['price'], product['shopper'], product['ads'], product['rank']]], columns=['구분', '브랜드', '키워드', '페이지수', '위치', '상품명', '상품ID', '가격', '셀러명', '광고아이디', '광고/오가닉순위']))
                                rank += 1
                            if len(mobile_product) > 0:
                                mobile_product['분류'] = '네이버_모바일'
                                result = mobile_product
                naverqq = pd.concat([naverqq, result])
            try:
                naverqq = naverqq[['위치','분류','상품명','상품ID','가격', '셀러명', '구분']]
                naverqq.columns = ['index', '채널','상품명', '상품ID', '가격', '판매자', '광고여부']
            except:
                naverqq = pd.DataFrame()
                errorlst.append({'브랜드': br,'채널':'네이버','키워드':brandkeyword[br][jojo]})

        #######
        # 쿠팡 시작
        #######
            def getPageString(url):
                data = requests.get(url, headers = headers)
                return data.content
            
            headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'no-cache',
            'cookie': 'mid=YfNn_wAEAAGcsrsw8WpRAvPXFgPo; ig_did=29AF9B8C-C62A-4EE5-BAC7-3C081EA9AB8F; ig_nrcb=1; csrftoken=W9GZ945eA65uZA87rBM7wnxZ8V4pj7wN; ds_user_id=1305778462; sessionid=1305778462%3AGNkC0wuSqS3mcZ%3A11; shbid="8520\0541305778462\0541674879758:01f7aceb492e0f47a3596c0a6dc1d9133e175142c2b6268267277abd1281e7dc2144372b"; shbts="1643343758\0541305778462\0541674879758:01f73164fb8aef28db280f397fceae005a5cbc050174dff1e5d3710a1c86b8b02c9e022f"; rur="VLL\0541305778462\0541674879768:01f70cac8366a361d7456f77ac0111584b70d57e0a8ad8b19b2c4fca58b7b9cad51b638c"',
            'pragma': 'no-cache',
            'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"macOS"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
            }
            print('쿠팡')

            def vendor(link):
                qq = link
                qq = 'https://www.coupang.com/vp/' + qq.split("/vm/")[-1]
                res1 = requests.get(qq, headers = headers).text
                soup1 = BeautifulSoup(res1, 'html.parser')
                tt = soup1.find_all('script')[4].text.split("exports.sdp = ")[-1].split(";\n        exports")[0]
                try:
                    pp = json.loads(tt)  
                    try:
                        return pp['vendor']['name']
                    except:
                        return '쿠팡'
                except:
                    return '추적 문제'


            search = keyword
            url = "https://m.coupang.com/nm/search?q={}".format(search)

            pageString = getPageString(url)
            soup = BeautifulSoup(pageString, "html.parser")
            # a = soup.find_all('div',{'class':'details'})
            a = soup.find_all('li',{'class':'plp-default__item'})
            pageone = []
            try:
                for data in a:
                    link = data.find('a')['href']
                    sku = link.split('/')[-1].split("?")[0]
                    try:
                        ad = data.find("span",{'class':'ad-badge-text'}).text
                    except AttributeError:
                        ad = data.find("span",{'class':'ad-badge-text'})
                    title = data.find('strong',{'class':'title'}).text
                    try:
                        discount = data.find("span", {'class':'percentage'}).text
                    except AttributeError:
                        discount = data.find("span", {'class':'percentage'})
                    try:
                        ori_price = data.find("strong",{'class':'price'}).text.split('원')[0]        
                    except AttributeError:        
                        ori_price = data.find("strong",{'class':'price'})
                    dis_price = data.find("div",{"class":"discount-price"}).find("strong").text
                    pageone.append({'상품명':title,'상품ID':sku,'할인율':discount,'정가':ori_price,'할인가':dis_price,'광고유무':ad,'링크':link})
                pageonedf = pd.DataFrame(pageone)

                pageonedf['판매자'] = pageonedf['링크'].apply(lambda x: vendor(x))
                pageonedf = pageonedf[['상품명', '상품ID', '할인율', '정가', '할인가', '광고유무', '판매자']]
                pageonedf['채널'] = '쿠팡'
                pageonedf = pageonedf[['채널','상품명', '상품ID', '할인가', '판매자','광고유무']]
                pageonedf.columns = ['채널','상품명', '상품ID', '가격', '판매자', '광고여부']
                coupang = pageonedf
            except:
                errorlst.append({'브랜드': br,'채널':'쿠팡','키워드':brandkeyword[br][jojo]})
                coupang = pd.DataFrame()

        #######
        # 옥션 시작
        #######
            # headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
            print('옥션')
            headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'no-cache',
            'cookie': 'mid=YfNn_wAEAAGcsrsw8WpRAvPXFgPo; ig_did=29AF9B8C-C62A-4EE5-BAC7-3C081EA9AB8F; ig_nrcb=1; csrftoken=W9GZ945eA65uZA87rBM7wnxZ8V4pj7wN; ds_user_id=1305778462; sessionid=1305778462%3AGNkC0wuSqS3mcZ%3A11; shbid="8520\0541305778462\0541674879758:01f7aceb492e0f47a3596c0a6dc1d9133e175142c2b6268267277abd1281e7dc2144372b"; shbts="1643343758\0541305778462\0541674879758:01f73164fb8aef28db280f397fceae005a5cbc050174dff1e5d3710a1c86b8b02c9e022f"; rur="VLL\0541305778462\0541674879768:01f70cac8366a361d7456f77ac0111584b70d57e0a8ad8b19b2c4fca58b7b9cad51b638c"',
            'pragma': 'no-cache',
            'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"macOS"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
            }

            def getPageString(url):
                data = requests.get(url, headers = headers)
                return data.content

            url = 'http://browse.auction.co.kr/m/search?keyword={}'.format(keyword)
            urlpageString = getPageString(url)
            soup = BeautifulSoup(urlpageString, "html.parser")
            soup
            a = soup.find_all("div",{'class':'component component--item_card type--general'})
            onepage = []
            try:
                for data in a:
                    title = data.find("span",{'class':'text--title'}).text
                    sku = data.find('a',{'class':'link--itemcard'})['href'].split("itemno")[-1].split('&data')[0]
                    price = data.find("strong",{'class':'text--price_seller'}).text
                    try:
                        seller = data.find('a',{'class':'link--shop'}).find("span",{'class':'text'}).text
                    except:
                        seller = data.find('a',{'class':'link--shop'}).find("span",{'class':'img'}).text
                    try:
                        ad = data.find('div',{'class':'section--component_title'}).text
                    except:
                        ad = data.find('div',{'class':'section--component_title'})
                    onepage.append({'상품명':title,'상품ID':sku,'가격':price,'판매자':seller,'광고':ad})
                df = pd.DataFrame(onepage)
                k = '광고'
                for i in range(len(df)):
                    if df.loc[i,'광고'] != '일반등록':
                        df.loc[i,'광고'] = k
                        df.loc[i,'상품ID'] = df.loc[i,'상품ID'].split("%3d")[-1]
                    else:
                        k = ''
                        df.loc[i,'광고'] = k
                    if df.loc[i,'광고'] == '':
                        df.loc[i,'상품ID'] = df.loc[i,'상품ID'].split("=")[-1]
                df['채널'] = '옥션'
                df = df[['채널','상품명', '상품ID', '가격', '판매자', '광고']]
                df.columns = ['채널','상품명', '상품ID', '가격', '판매자', '광고여부']
                auction = df
            except:
                errorlst.append({'브랜드': br,'채널':'옥션','키워드':brandkeyword[br][jojo]})
                auction = pd.DataFrame()
        #######
        # 지마켓 시작
        #######
            # headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
            print('지마켓')
            url = 'http://browse.gmarket.co.kr/m/search?keyword={}'.format(keyword)
            # url = 'http://browse.gmarket.co.kr/m/search?keyword=%EC%A0%90%ED%94%84%EC%97%85%ED%95%98%EC%9D%B4%EC%A0%95'
            urlpageString = getPageString(url)
            soup = BeautifulSoup(urlpageString, "html.parser")
            soup
            a = soup.find_all("div",{'class':'box__component box__component-itemcard box__component-itemcard--general'})
            onepage = []
            try:
                for data in a:
                    title = data.find("span",{'class':'text__item'}).text
                    sku = data.find('a',{'class':'link__item'})['href'].split("goodscode=")[-1]
                    link = data.find('a',{'class':'link__item'})['href']
                    price = data.find("strong",{'class':'text text__value'}).text
                    try:
                        seller = data.find('a',{'class':'link__shop'}).find("span",{'class':'text__seller'}).text
                    except:
                        seller = data.find('a',{'class':'link__shop'}).find("span",{'class':'text__seller'})
                    try:
                        ad = data.find('p',{'class':'text__title'}).text
                    except:
                        ad = data.find('p',{'class':'text__title'})
                    onepage.append({'상품명':title,'상품ID':str(sku),'가격':price,'판매자':seller,'광고':ad})
                df = pd.DataFrame(onepage)
                k = '광고'
                for i in range(len(df)):
                    if df.loc[i,'광고'] != '일반상품':
                        df.loc[i,'광고'] = k
                        df.loc[i,'상품ID'] = df.loc[i,'상품ID'].split("%3d")[-1]
                    else:
                        k = ''
                        df.loc[i,'광고'] = k
                    if df.loc[i,'광고'] == '':
                        df.loc[i,'상품ID'] = df.loc[i,'상품ID'].split("=")[-1]

                df['채널'] = '지마켓'
                df = df[['채널','상품명', '상품ID', '가격', '판매자', '광고']]
                df.columns = ['채널','상품명', '상품ID', '가격', '판매자', '광고여부']
                gmarket = df
            except:
                errorlst.append({'브랜드': br,'채널':'지마켓','키워드':brandkeyword[br][jojo]})
                gmarket = pd.DataFrame()

        #######
        # 11번가 시작
        #######
            print('11번가')
            keyword1 = keyword
            ll = (urllib.parse.quote(keyword1,safe='')).split('%')
            qq = ''
            for l in ll:
                qq += l
                qq += '%25'
            keyword = qq[:-3]
            keyword

            site_url = 'http://search.11st.co.kr/MW/search?searchKeyword={}'.format(keyword)

            res = requests.get(site_url).text
            soup = BeautifulSoup(res,'html.parser')
            a = soup.findAll('script')[0].text.split("window.N11ST.model =  ")[-1].split("\n    console.time(\'HTML to Render\')")[0][:-1].split(';\n\n')[0]
            a = a.replace(';\n        console.log(\'SCHFE-10-3\')\n   ','')
            b = json.loads(a)
            k = []
            try:
                for pp in range(len(b['data'])):
                    try:
                        for info in b['data'][pp]['items']:
                            title = info['title']
                            link = info['linkUrl']
                            sku = str(info['id'])
                            price = info['finalPrc']
                            ad = info['logData']['area']
                            if 'https://catalog.11st.co.kr/MW/Catalog/catalogDetail.tmall?catalogNo' in link:
                                sku = link
                                CtgrNo = link.split('catalogNo=')[-1].split("&")[-1].split("=")[-1]
                            k.append({'상품명':title,'상품ID':sku,'가격':price,'광고':ad})
                    except:
                        pass
                adf = pd.DataFrame(k)
                adf['판매자'] = ''
                adf['광고여부'] = ''
                for i in range(len(adf)):
                    if 'https://catalog.11st.co.kr/MW/Catalog/catalogDetail.tmall?catalogNo' in str(adf.loc[i,'상품ID']):
                        catalogNo = adf.loc[i,'상품ID'].split('catalogNo=')[-1].split("&")[0]
                        CtgrNo = adf.loc[i,'상품ID'].split('catalogNo=')[-1].split("&")[-1].split("=")[-1]
                        url = 'https://catalog.11st.co.kr/api/Catalog/ajaxDetailList.tmall?catalogNo={}&sellerNos=&trTypeCd=MAS51&trCtgrNo={}'.format(catalogNo, CtgrNo)
                        res1 = requests.get(url)
                        soup1 = res1.json()
                        try:
                            seller = soup1['data'][1]['priceCompare']['packageInfoList'][0]['prdList'][0]['memNm']
                            sku = soup1['data'][1]['priceCompare']['packageInfoList'][0]['prdList'][0]['prdNo']
                        except:
                            seller = soup1['data'][1]['priceCompare']['prdList'][0]['memNm']
                            sku = soup1['data'][1]['priceCompare']['prdList'][0]['prdNo']
                        adf.loc[i,'상품ID'] = str(sku)
                        adf.loc[i,'판매자'] = seller
                        if adf.loc[i,'광고'] == 'catalog_compare':
                            adf.loc[i,'광고여부'] = '가격비교 컬럼'
                        else:
                            adf.loc[i,'광고여부'] = '가격비교'
                    else:
                        url = 'https://www.11st.co.kr/products/{}?trTypeCd=MAS24&trCtgrNo={}'.format(adf.loc[i,'상품ID'],CtgrNo)
                        res1 = requests.get(url).text
                        res1 = BeautifulSoup(res1,'html.parser')
                        seller = res1.find("h1", {'class':"c_product_store_title"}).find("a").text.replace(" ",'').replace("\n",'')        
                        adf.loc[i,'판매자'] = seller
                        if 'focus' in adf.loc[i,'광고'] or 'plus' in adf.loc[i,'광고']:
                            adf.loc[i,'광고여부'] = '광고'

                adf['채널'] = '11번가'
                adf = adf[['채널','상품명', '상품ID', '가격', '판매자', '광고여부']]
                street = adf
            except:
                errorlst.append({'브랜드': br,'채널':'11번가','키워드':brandkeyword[br][jojo]})
                street = pd.DataFrame()
            keyword = brandkeyword[br][jojo]
    
        ######
        ##티몬
        ######
            print('티몬')
            headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'no-cache',
            'referer': 'http://m.search.tmon.co.kr/search?',
            'cookie': 'mid=YfNn_wAEAAGcsrsw8WpRAvPXFgPo; ig_did=29AF9B8C-C62A-4EE5-BAC7-3C081EA9AB8F; ig_nrcb=1; csrftoken=W9GZ945eA65uZA87rBM7wnxZ8V4pj7wN; ds_user_id=1305778462; sessionid=1305778462%3AGNkC0wuSqS3mcZ%3A11; shbid="8520\0541305778462\0541674879758:01f7aceb492e0f47a3596c0a6dc1d9133e175142c2b6268267277abd1281e7dc2144372b"; shbts="1643343758\0541305778462\0541674879758:01f73164fb8aef28db280f397fceae005a5cbc050174dff1e5d3710a1c86b8b02c9e022f"; rur="VLL\0541305778462\0541674879768:01f70cac8366a361d7456f77ac0111584b70d57e0a8ad8b19b2c4fca58b7b9cad51b638c"',
            'pragma': 'no-cache',
            'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"macOS"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
            }
            keyword = brandkeyword[br][jojo]
            url = 'http://m.search.tmon.co.kr/api/search/v4/deals?keyword={}'.format(keyword)
            urlpageString = requests.get(url,headers=headers)
            soup = urlpageString.json()
            a = soup['data']['searchDeals']
            prolst = []
            i = 1
            try:
                for data in a:
                    if data['searchDealResponse']['searchInfo']['isOptionDeal'] == False:
                        sku = data['searchDealResponse']['searchInfo']['id']
                        name = data['searchDealResponse']['dealInfo']['titleName']
                        price = data['searchDealResponse']['dealInfo']['priceInfo']['price']
                        partnerno = data['searchDealResponse']['dealInfo']['partnerNo']
                        tt = 'http://mobile.tmon.co.kr/api/detail/direct/v1/detailapi/api/partner/info/{}?mainDealNo={}'.format(partnerno,sku)
                        urlpageString1 = requests.get(tt,headers=headers)
                        seller = BeautifulSoup(urlpageString1.content,'html.parser').find("ecaccountnm").text

                        try:
                            ad = data['searchDealResponse']['dealInfo']['admonUrl']
                            ad = '광고'
                        except:
                            ad = ''
                        prolst.append({'순위':i, '채널':'티몬','상품ID':sku, '상품명':name, '가격':price,'광고':ad, '판매자':seller})
                        i += 1
                prodf = pd.DataFrame(prolst)
                tmondf = prodf
                tmondf.columns = ['index', '채널', '상품ID', '상품명', '가격', '광고여부', '판매자']
                tmondf = tmondf[['채널','index','상품명', '상품ID', '가격', '판매자', '광고여부']]
            except:
                errorlst.append({'브랜드': br,'채널':'티몬','키워드':brandkeyword[br][jojo]})
                tmondf = pd.DataFrame()
                
        #####
        ##위메프
        #####
            print('위메프')
            keyword = brandkeyword[br][jojo]
            url = 'https://search.wemakeprice.com/api/wmpsearch/api/v3.0/wmp-search/search.json?searchType=DEFAULT&search_cate=top&keyword={}'.format(keyword)
            urlpageString = requests.get(url)
            soup = urlpageString.json()
            #### 광고
            adlst = []
            ### 광고용
            try:
                k = soup['data']['adTargetClick']['data']['deals']
            except:
                print('광고없음')
            ###일반용
            try:
                a = soup['data']['deals']
            except:
                print('일반 없음')
            try:
                for data in k:
                    channel = '위메프'
                    name = data['dispNm']
                    if data['discountPrice']==None:
                        price = data['salePrice']
                    else:
                        price = data['discountPrice']
                    sku =   data['link']['value']
                    try:
                        conditionid = data['link']['option']['query']['buyingConditionId']
                        catalognum = data['catalog']['catalogNo']
                        tt = 'https://front.wemakeprice.com/api/cataloglisting/v2/catalog/{}.json?buycdtId={}'.format(catalognum, conditionid)
                        urlpageString1 = requests.get(tt)
                        soup1 = urlpageString1.json()
                        seller = soup1['data']['prodLists'][0]['list'][0]['sellerNm']
                    except:
                        tt = 'https://front.wemakeprice.com/product/{}'.format(sku)
                        def getPageString(url):
                            data = requests.get(url, headers = headers)
                            return data.content

                        urlpageString = getPageString(tt)
                        soup = BeautifulSoup(urlpageString, "html.parser")
                        seller = soup.find('body').find_all('table',{'class':'table_info'})[5].find('span').text

                    adlst.append({'상품명':name,'상품ID':sku,'가격':price,'판매자':seller,'광고여부':'광고'})

                u = pd.DataFrame(adlst)

                ### 일반

                prolst = []
                for data in a:
                    channel = '위메프'
                    name = data['dispNm']
                    if data['discountPrice']==None:
                        price = data['salePrice']
                    else:
                        price = data['discountPrice']
                    sku =   data['link']['value']
                    try:
                        conditionid = data['link']['option']['query']['buyingConditionId']
                        catalognum = data['catalog']['catalogNo']
                        tt = 'https://front.wemakeprice.com/api/cataloglisting/v2/catalog/{}.json?buycdtId={}'.format(catalognum, conditionid)
                        urlpageString1 = requests.get(tt)
                        soup1 = urlpageString1.json()
                        seller = soup1['data']['prodLists'][0]['list'][0]['sellerNm']
                    except:
                        tt = 'https://front.wemakeprice.com/product/{}'.format(sku)
                        def getPageString(url):
                            data = requests.get(url, headers = headers)
                            return data.content

                        urlpageString = getPageString(tt)
                        soup = BeautifulSoup(urlpageString, "html.parser")
                        try:
                            seller = soup.find('body').find_all('table',{'class':'table_info'})[3].find('span').text
                            if '배송비' in seller:
                                seller = ''
                        except:
                            seller = ''
                    prolst.append({'상품명':name,'상품ID':sku,'가격':price,'판매자':seller,'광고여부':''})

                prodf = pd.DataFrame(prolst)
                wemakedf = pd.concat([u, prodf])
                wemakedf.reset_index(inplace=True)
                wemakedf = wemakedf[['상품명', '상품ID', '가격', '판매자', '광고여부']]
                wemakedf.reset_index(inplace=True)
                wemakedf['index'] = wemakedf['index'].apply(lambda x: x+1)
                wemakedf['채널'] = '위메프'
                wemakedf = wemakedf[['채널','index','상품명', '상품ID', '가격', '판매자', '광고여부']]
            except:
                errorlst.append({'브랜드': br,'채널':'위메프','키워드':brandkeyword[br][jojo]})
                wemakedf = pd.DataFrame()

            street.reset_index(inplace=True)
            auction.reset_index(inplace=True)
            gmarket.reset_index(inplace=True)
            coupang.reset_index(inplace=True)
            keyworddf = pd.concat([street, auction, gmarket, coupang])
            keyworddf['index'] = keyworddf['index'].apply(lambda x: x+1)
            keyworddf = pd.concat([keyworddf,naverqq,tmondf,wemakedf])
            keyworddf['날짜'] = datetime.now().strftime("%y-%m-%d")
            keyworddf = keyworddf[['날짜','채널','index','상품명', '상품ID', '가격', '판매자', '광고여부']]
            keyworddf.columns = ['날짜','채널','순위','상품명', '상품ID', '가격', '판매자', '광고여부']

                

            keyworddf.to_excel(writer, sheet_name = '{}'.format(keyword), index = False)